In [1]:
import os
import random
import sqlite3
import sys
import time
from datetime import date, datetime, timedelta
import pandas as pd
import re 

from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
from nba_api.stats.endpoints import boxscoretraditionalv3
import toml
import requests
from tqdm import tqdm
from sbrscrape import Scoreboard
import numpy as np



## Section 1: Defining Headers and Helper Functions

In [2]:
data_headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.nba.com/',
    'Connection': 'keep-alive'
}


def to_data_frame(data):
    try:
        data_list = data[0]
    except Exception as e:
        print(e)
        return pd.DataFrame(data={})
    return pd.DataFrame(data=data_list.get('rowSet'), columns=data_list.get('headers'))

def get_json_data(url):
    raw_data = requests.get(url, headers=data_headers)
    try:
        json = raw_data.json()
    except Exception as e:
        print(e)
        return {}
    return json.get('resultSets')


def get_date(date_string):
    year1,month,day = re.search(r'(\d+)-\d+-(\d\d)(\d\d)', date_string).groups()
    year = year1 if int(month) > 8 else int(year1) + 1
    return datetime.strptime(f"{year}-{month}-{day}", '%Y-%m-%d')

def generate_seasons(from_year, to_year):
    seasons = []
    for year in range(int(from_year), int(to_year) + 1):
        season_str = f"{year}-{str((year + 1) % 100).zfill(2)}"
        seasons.append(season_str)
    return seasons

# ## Used in creating/updating teams database
# def get_last_processed_date(table_name, connection):
#     query = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'"
#     cursor = connection.cursor()
#     cursor.execute(query)
#     result = cursor.fetchone()

#     if result:
#         # The table exists; get the most recent date
#         query = f"SELECT MAX(Date) FROM '{table_name}'"
#         cursor.execute(query)
#         last_date = cursor.fetchone()[0]
#         if last_date:
#             return datetime.strptime(last_date, "%Y-%m-%d").date()
#     return None  # Table doesn't exist or has no data

def get_last_processed_date(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_list = cursor.fetchall()
    
    table_list = list(table_list)
    date_objects = [datetime.strptime(date[0], "%Y-%m-%d").date() for date in table_list]
    max_date = max(date_objects)

    return max_date

url = 'https://stats.nba.com/stats/' \
      'leaguedashteamstats?Conference=&' \
      'DateFrom=10%2F01%2F{2}&DateTo={0}%2F{1}%2F{3}' \
      '&Division=&GameScope=&GameSegment=&LastNGames=0&' \
      'LeagueID=00&Location=&MeasureType=Base&Month=0&' \
      'OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&' \
      'PerMode=PerGame&Period=0&PlayerExperience=&' \
      'PlayerPosition=&PlusMinus=N&Rank=N&' \
      'Season={4}' \
      '&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&' \
      'StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='


## Section 2: ML Model Database Building/Updating 

In [48]:
## This chunk was used to drop duplicate tables based on desired date range 
## TODO Turn this into helper function to be used in the future for cleaning needs 

current_datetime = datetime.now() - timedelta(days= 1)
current_datetime = current_datetime.date()
date_tracker = current_datetime
date_tracker = datetime.strptime("2023-10-24", "%Y-%m-%d").date()
end_date = datetime.strptime("2025-4-14", "%Y-%m-%d").date()


sys.path.insert(1, os.path.join(sys.path[0], '..'))

config = toml.load("config.toml")
con = sqlite3.connect("Data/teams.sqlite")

## Start date 1/24/25, end date 4/14/25


while date_tracker<=end_date:
    print(date_tracker)
    cursor = con.cursor()
    query =  f"SELECT name FROM sqlite_master WHERE type='table' AND name='{date_tracker}'"
    cursor.execute(query)
    result = cursor.fetchone()
    if result is not None:
        print("Table Dropped")
        dropquery = f"DROP TABLE '{date_tracker}'"
        cursor.execute(dropquery)

    else:
        print("Table does not exist")
    date_tracker = date_tracker + timedelta(days=1)
con.close()

2023-10-24
Table Dropped
2023-10-25
Table Dropped
2023-10-26
Table Dropped
2023-10-27
Table Dropped
2023-10-28
Table Dropped
2023-10-29
Table Dropped
2023-10-30
Table Dropped
2023-10-31
Table Dropped
2023-11-01
Table Dropped
2023-11-02
Table Dropped
2023-11-03
Table Dropped
2023-11-04
Table Dropped
2023-11-05
Table Dropped
2023-11-06
Table Dropped
2023-11-07
Table Dropped
2023-11-08
Table Dropped
2023-11-09
Table Dropped
2023-11-10
Table Dropped
2023-11-11
Table Dropped
2023-11-12
Table Dropped
2023-11-13
Table Dropped
2023-11-14
Table Dropped
2023-11-15
Table Dropped
2023-11-16
Table Dropped
2023-11-17
Table Dropped
2023-11-18
Table Dropped
2023-11-19
Table Dropped
2023-11-20
Table Dropped
2023-11-21
Table Dropped
2023-11-22
Table Dropped
2023-11-23
Table Dropped
2023-11-24
Table Dropped
2023-11-25
Table Dropped
2023-11-26
Table Dropped
2023-11-27
Table Dropped
2023-11-28
Table Dropped
2023-11-29
Table Dropped
2023-11-30
Table Dropped
2023-12-01
Table Dropped
2023-12-02
Table Dropped


## Section 2A: Pulling Team Season Stats by Date

DB: 'teams.sqlite'

Table Naming Scheme: "YYYY-MM-DD"

Tables contain season averages in various stats back to 2007

In [49]:
## TODO Cleanup this code and double check teams.sqlite file, believe this is just lookup table
## Took 174min to run through 1/24/25

## teams Database file 
## Creates table for each day of the regular season with team stats

sys.path.insert(1, os.path.join(sys.path[0], '..'))

config = toml.load("config.toml")
con = sqlite3.connect("Data/teams.sqlite")

## For each season in the list, go through each month and day and pull the game data

## TODO Add code to check latest entry in table and update records

for key, value in config['get-data'].items():
    start_date = datetime.strptime(value['start_date'], "%Y-%m-%d").date()
    end_date = datetime.strptime(value['end_date'], "%Y-%m-%d").date()

    ## Check current date time (using previous day to not create in cases where games haven't concluded yet) and don't create tables for future
    current_datetime = datetime.now() - timedelta(days= 1)

    ## TODO Test if this section works. Need to clean up table to delete dupe records created for future dates. 
    if end_date > current_datetime.date():
        end_date = current_datetime.date()


    last_processed_date = get_last_processed_date(con)

    if last_processed_date:
        date_pointer = last_processed_date #+ timedelta(days=1)
    else:
        # If no data exists, start from the configured start_date
        date_pointer = start_date

    while date_pointer <= end_date:
        print("Getting data: ", date_pointer)

        raw_data = get_json_data(
            url.format(date_pointer.month, date_pointer.day, value['start_year'], date_pointer.year, key))
        df = to_data_frame(raw_data)

        date_pointer = date_pointer + timedelta(days=1)

        df['Date'] = str(date_pointer)

        df.to_sql(date_pointer.strftime("%Y-%m-%d"), con, if_exists="replace")

        time.sleep(random.randint(1, 3))


con.close()

Getting data:  2023-06-13
Getting data:  2023-06-14
Getting data:  2023-06-15
Getting data:  2023-06-16
Getting data:  2023-06-17
Getting data:  2023-06-18
Getting data:  2023-06-19
Getting data:  2023-06-20
Getting data:  2023-06-21
Getting data:  2023-06-22
Getting data:  2023-06-23
Getting data:  2023-06-24
Getting data:  2023-06-25
Getting data:  2023-06-26
Getting data:  2023-06-27
Getting data:  2023-06-28
Getting data:  2023-06-29
Getting data:  2023-06-30
Getting data:  2023-07-01
Getting data:  2023-07-02
Getting data:  2023-07-03
Getting data:  2023-07-04
Getting data:  2023-07-05
Getting data:  2023-07-06
Getting data:  2023-07-07
Getting data:  2023-07-08
Getting data:  2023-07-09
Getting data:  2023-07-10
Getting data:  2023-07-11
Getting data:  2023-07-12
Getting data:  2023-07-13
Getting data:  2023-07-14
Getting data:  2023-07-15
Getting data:  2023-07-16
Getting data:  2023-07-17
Getting data:  2023-07-18
Getting data:  2023-07-19
Getting data:  2023-07-20
Getting data

## Section 2B: Pulling Odds Data 

DB: OddsData.sqlite

ADD FURTHER DESCRIPTION

In [42]:
sportsbook = 'fanduel'
df_data = []

config = toml.load("config.toml")

con = sqlite3.connect("Data/oddsData.sqlite")

for key, value in config['get-odds-data'].items():
    date_pointer = datetime.strptime(value['start_date'], "%Y-%m-%d").date()
    end_date = datetime.strptime(value['end_date'], "%Y-%m-%d").date()
    teams_last_played = {}

    current_datetime = datetime.now() - timedelta(days= 1)

    if end_date > current_datetime.date():
        end_date = current_datetime.date()
    

    while date_pointer <= end_date:
        print("Getting odds data: ", date_pointer)
        sb = Scoreboard(date=date_pointer)

        ## Check if date had any games played and skip if not  
        if not hasattr(sb, "games"):
            date_pointer = date_pointer + timedelta(days=1)
            continue

        for game in sb.games:
            if game['home_team'] not in teams_last_played:
                teams_last_played[game['home_team']] = date_pointer
                home_games_rested = timedelta(days=7)  # start of season, big number
            else:
                current_date = date_pointer
                home_games_rested = current_date - teams_last_played[game['home_team']]
                teams_last_played[game['home_team']] = current_date

            if game['away_team'] not in teams_last_played:
                teams_last_played[game['away_team']] = date_pointer
                away_games_rested = timedelta(days=7)  # start of season, big number
            else:
                current_date = date_pointer
                away_games_rested = current_date - teams_last_played[game['away_team']]
                teams_last_played[game['away_team']] = current_date

            try:
                df_data.append({
                    'Date': date_pointer,
                    'Home': game['home_team'],
                    'Away': game['away_team'],
                    'OU': game['total'][sportsbook],
                    'Spread': game['away_spread'][sportsbook],
                    'ML_Home': game['home_ml'][sportsbook],
                    'ML_Away': game['away_ml'][sportsbook],
                    'Points': game['away_score'] + game['home_score'],
                    'Win_Margin': game['home_score'] - game['away_score'],
                    'Days_Rest_Home': home_games_rested.days,
                    'Days_Rest_Away': away_games_rested.days
                })
            except KeyError:
                print(f"No {sportsbook} odds data found for game: {game}")

        date_pointer = date_pointer + timedelta(days=1)
        time.sleep(random.randint(1, 3))

    df = pd.DataFrame(df_data, )
    df.to_sql('odds_'+key+'_new', con, if_exists="replace")
con.close()

Getting odds data:  2023-10-23
Getting odds data:  2023-10-24
Getting odds data:  2023-10-25
Getting odds data:  2023-10-26
Getting odds data:  2023-10-27
Getting odds data:  2023-10-28
Getting odds data:  2023-10-29
Getting odds data:  2023-10-30
Getting odds data:  2023-10-31
Getting odds data:  2023-11-01
Getting odds data:  2023-11-02
Getting odds data:  2023-11-03
Getting odds data:  2023-11-04
Getting odds data:  2023-11-05
Getting odds data:  2023-11-06
Getting odds data:  2023-11-07
Getting odds data:  2023-11-08
Getting odds data:  2023-11-09
Getting odds data:  2023-11-10
Getting odds data:  2023-11-11
Getting odds data:  2023-11-12
Getting odds data:  2023-11-13
Getting odds data:  2023-11-14
Getting odds data:  2023-11-15
Getting odds data:  2023-11-16
Getting odds data:  2023-11-17
Getting odds data:  2023-11-18
Getting odds data:  2023-11-19
Getting odds data:  2023-11-20
Getting odds data:  2023-11-21
Getting odds data:  2023-11-22
Getting odds data:  2023-11-23
Getting 

## Section 2C: Joining Odds and Team Data

DB: 'ML_master_dataset.sqlite'

ADD FURTHER DESCRIPTION

In [50]:
from Util.Dictionaries import team_index_07, team_index_08, team_index_12, team_index_13, team_index_14, \
    team_index_current

df = pd.DataFrame
scores = []
win_margin = []
OU = []
OU_Cover = []
games = []
days_rest_away = []
days_rest_home = []
teams_con = sqlite3.connect("Data/teams.sqlite")
odds_con = sqlite3.connect("Data/OddsData.sqlite")

for key, value in config['create-games'].items():
    print(key)
    odds_df = pd.read_sql_query(f"select * from \"odds_{key}_new\"", odds_con, index_col="index")
    team_table_str = key
    year_count = 0
    season = key

    for row in odds_df.itertuples():
        home_team = row[2]
        away_team = row[3]

        date = row[1]

        team_df = pd.read_sql_query(f"select * from \"{date}\"", teams_con, index_col="index")
        if len(team_df.index) == 30:
            scores.append(row[8])
            OU.append(row[4])
            days_rest_home.append(row[10])
            days_rest_away.append(row[11])
            if row[9] > 0:
                win_margin.append(1)
            else:
                win_margin.append(0)

            if row[8] < row[4]:
                OU_Cover.append(0)
            elif row[8] > row[4]:
                OU_Cover.append(1)
            elif row[8] == row[4]:
                OU_Cover.append(2)

            if season == '2007-08':
                home_team_series = team_df.iloc[team_index_07.get(home_team)]
                away_team_series = team_df.iloc[team_index_07.get(away_team)]
            elif season == '2008-09' or season == "2009-10" or season == "2010-11" or season == "2011-12":
                home_team_series = team_df.iloc[team_index_08.get(home_team)]
                away_team_series = team_df.iloc[team_index_08.get(away_team)]
            elif season == "2012-13":
                home_team_series = team_df.iloc[team_index_12.get(home_team)]
                away_team_series = team_df.iloc[team_index_12.get(away_team)]
            elif season == '2013-14':
                home_team_series = team_df.iloc[team_index_13.get(home_team)]
                away_team_series = team_df.iloc[team_index_13.get(away_team)]
            elif season == '2022-23' or season == '2023-24' or season == '2024-25':
                home_team_series = team_df.iloc[team_index_current.get(home_team)]
                away_team_series = team_df.iloc[team_index_current.get(away_team)]
            else:
                try:
                    home_team_series = team_df.iloc[team_index_14.get(home_team)]
                    away_team_series = team_df.iloc[team_index_14.get(away_team)]
                except Exception as e:
                    print(home_team)
                    raise e
            game = pd.concat([home_team_series, away_team_series.rename(
                index={col: f"{col}.1" for col in team_df.columns.values}
            )])
            games.append(game)
odds_con.close()
teams_con.close()
season = pd.concat(games, ignore_index=True, axis=1)
season = season.T
frame = season.drop(columns=['TEAM_ID', 'TEAM_ID.1'])
frame['Score'] = np.asarray(scores)
frame['Home-Team-Win'] = np.asarray(win_margin)
frame['OU'] = np.asarray(OU)
frame['OU-Cover'] = np.asarray(OU_Cover)
frame['Days-Rest-Home'] = np.asarray(days_rest_home)
frame['Days-Rest-Away'] = np.asarray(days_rest_away)
# fix types
for field in frame.columns.values:
    if 'TEAM_' in field or 'Date' in field or field not in frame:
        continue
    frame[field] = frame[field].astype(float)
con = sqlite3.connect("Data/ML_master_dataset.sqlite")
frame.to_sql("dataset_2012-24_new", con, if_exists="replace")
con.close()

2012-13
2013-14
2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21
2021-22
2022-23
2023-24
2024-25


## Section 3: Generating/Updating Player Game Log DB

In [51]:
## TEST CODE

from nba_api.stats.endpoints import playergamelogs

## Example code for querying player game log data. 
test= playergamelogs.PlayerGameLogs(player_id_nullable = '203076', season_nullable= ['2022-23','2023-24'], season_type_nullable='Regular Season')

test = test.get_data_frames()[0]
print(test)

## TEST 2 Looking ad advanced box score from game id
from nba_api.stats.endpoints import boxscoreadvancedv3

id = '0022301195'
test_adv= boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=id) 

print(test_adv.get_data_frames()[0])


   SEASON_YEAR  PLAYER_ID    PLAYER_NAME NICKNAME     TEAM_ID  \
0      2022-23     203076  Anthony Davis  Anthony  1610612747   
1      2022-23     203076  Anthony Davis  Anthony  1610612747   
2      2022-23     203076  Anthony Davis  Anthony  1610612747   
3      2022-23     203076  Anthony Davis  Anthony  1610612747   
4      2022-23     203076  Anthony Davis  Anthony  1610612747   
5      2022-23     203076  Anthony Davis  Anthony  1610612747   
6      2022-23     203076  Anthony Davis  Anthony  1610612747   
7      2022-23     203076  Anthony Davis  Anthony  1610612747   
8      2022-23     203076  Anthony Davis  Anthony  1610612747   
9      2022-23     203076  Anthony Davis  Anthony  1610612747   
10     2022-23     203076  Anthony Davis  Anthony  1610612747   
11     2022-23     203076  Anthony Davis  Anthony  1610612747   
12     2022-23     203076  Anthony Davis  Anthony  1610612747   
13     2022-23     203076  Anthony Davis  Anthony  1610612747   
14     2022-23     203076

Framework for building NBA game log DB 

Task #1 
Create dictionary of player IDs (designate active v inactive) DONE

Task #2 
Use dictionary to go through by year and by player and add game logs to SQLite DB DONE
    Start with regular season and then add playoffs (use playergamelogs function)
    Expand with advanced stats 

Expand with Career/season stats page

In [5]:
## TODO Need to add checks for when new players are added to DB
## Current player count is 4940

from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo

# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
print('Number of players fetched: {}'.format(len(nba_players)))
player_id_df = pd.DataFrame(nba_players)




common_player_con = sqlite3.connect("Data/CommonPlayerInfo.sqlite")

with common_player_con:
    common_player_con.execute("""
        CREATE TABLE IF NOT EXISTS CommonPlayerInfo (
            PERSON_ID INTEGER, 
            FIRST_NAME TEXT, 
            LAST_NAME TEXT, 
            DISPLAY_FIRST_LAST TEXT, 
            DISPLAY_LAST_COMMA_FIRST TEXT, 
            DISPLAY_FI_LAST TEXT, 
            PLAYER_SLUG TEXT, 
            BIRTHDATE DATE, 
            SCHOOL TEXT, 
            COUNTRY TEXT, 
            LAST_AFFILIATION TEXT, 
            HEIGHT TEXT, 
            WEIGHT INTEGER, 
            SEASON_EXP INTEGER, 
            JERSEY TEXT, 
            POSITION TEXT, 
            ROSTERSTATUS TEXT, 
            aTEAM_ID INTEGER, 
            TEAM_NAME TEXT, 
            TEAM_ABBREVIATION TEXT, 
            TEAM_ID INTEGER, 
            TEAM_CODE TEXT, 
            TEAM_CITY TEXT, 
            PLAYERCODE INTEGER, 
            FROM_YEAR INTEGER, 
            TO_YEAR INTEGER, 
            DLEAGUE_FLAG TEXT, 
            NBA_FLAG TEXT, 
            GREATEST_75_FLAG TEXT, 
            GAMES_PLAYED_FLAG TEXT, 
            GAMES_PLAYED_CURRENT_SEASON_FLAG TEXT, 
            DRAFT_YEAR INTEGER, 
            DRAFT_ROUND INTEGER, 
            DRAFT_NUMBER INTEGER,
            PRIMARY KEY (PERSON_ID)
        )
    """)

#["PERSON_ID INTEGER, FIRST_NAME, LAST_NAME, DISPLAY_FIRST_LAST, DISPLAY_LAST_COMMA_FIRST, DISPLAY_FI_LAST, PLAYER_SLUG, BIRTHDATE, SCHOOL, COUNTRY, LAST_AFFILIATION, HEIGHT, WEIGHT, SEASON_EXP, JERSEY, POSITION, ROSTERSTATUS, aTEAM_ID, TEAM_NAME, TEAM_ABBREVIATION, TEAM_CODE, TEAM_CITY, PLAYERCODE, FROM_YEAR, TO_YEAR, DLEAGUE_FLAG, NBA_FLAG, GAMES_PLAYED_FLAG, DRAFT_YEAR, DRAFT_ROUND, DRAFT_NUMBER"]


## TODO change from creating one df iteratively to add to DB iteratively 
for id in player_id_df['id']:
    try:
        common_player_df = commonplayerinfo.CommonPlayerInfo(player_id=id)
        common_player_df = common_player_df.get_data_frames()[0]
  
        common_player_df.to_sql("CommonPlayerInfo", common_player_con, if_exists="append", index=False)
        print(f"Processed Player: {id}")
        time.sleep(random.random()+1)
    
    #df.to_sql('odds_'+key+'_new', con, if_exists="replace")
    except Exception as e:
        print(f"Error processing player ID {id}: {e}")

common_player_con.close()


Number of players fetched: 4940
0         76001
1         76002
2         76003
3            51
4          1505
         ...   
4935    1627790
4936      78647
4937      78648
4938    1627826
4939      78650
Name: id, Length: 4940, dtype: int64
Processed Player: 76001
Processed Player: 76002
Processed Player: 76003
Processed Player: 51
Processed Player: 1505
Processed Player: 949
Processed Player: 76005
Processed Player: 76006
Processed Player: 76007
Processed Player: 203518
Processed Player: 1630173
Processed Player: 101165
Processed Player: 76008
Processed Player: 76009
Processed Player: 76010
Processed Player: 203112
Processed Player: 76011
Processed Player: 76012
Processed Player: 200801
Processed Player: 1629121
Processed Player: 203919
Processed Player: 149
Processed Player: 203500
Processed Player: 912
Processed Player: 1628389
Processed Player: 1629061
Processed Player: 76015
Processed Player: 202399
Processed Player: 201167
Processed Player: 1630534
Processed Player: 200772
Pr

In [6]:
common_player_con = sqlite3.connect("Data/CommonPlayerInfo.sqlite")

cursor = common_player_con.cursor()

try:
    cursor.execute("ALTER TABLE CommonPlayerInfo ADD COLUMN GamesQueried INTEGER DEFAULT 0")
    cursor.execute("ALTER TABLE CommonPlayerInfo ADD COLUMN DateLastQueried TEXT")  # SQLite stores dates as TEXT
    print("Columns added successfully.")
except sqlite3.OperationalError as e:
    print("Error adding columns (they may already exist):", e)

common_player_con.commit()
common_player_con.close()



Columns added successfully.


In [38]:
from nba_api.stats.endpoints import playergamelogs

### Need to relabel summer league games based on game ID
### Need to create a basic framework for adding on advanced stats to this base table. 
### Should add master game log table to database. 

con = sqlite3.connect("Data/CommonPlayerInfo.sqlite")

cur = con.cursor()

cur.execute("SELECT PERSON_ID, FROM_YEAR, TO_YEAR, GamesQueried, DateLastQueried FROM CommonPlayerInfo")

player_id_years = cur.fetchall()



for row in player_id_years:

    if row[3] == 2:
        print("Already Collected" + str(row))
        continue

    if row[1] == None:
        print("No Year Available"+ str(row))
        continue
    print(row) 

    id = row[0]
    seasons = generate_seasons(row[1], row[2])
    player_data = pd.DataFrame()

    ### Update DB to show that games for this player have been queried
    cur.execute("UPDATE CommonPlayerInfo SET GamesQueried = 2 WHERE PERSON_ID = ?", (id,))

    for season in seasons:
        
        ##test= playergamelogs.PlayerGameLogs(player_id_nullable = id, season_nullable= season, season_type_nullable='Regular Season')
        test= playergamelogs.PlayerGameLogs(player_id_nullable = id, season_nullable= season, season_type_nullable='Playoffs')
        
        player_season = test.get_data_frames()[0]
        
        ##player_season['Season_Type'] = 'Regular Season'
        player_season['Season_Type'] = 'Playoffs'

        player_data = pd.concat([player_data, player_season], ignore_index=True)
        time.sleep(0.6)

        #print(player_season.head())
    
    #player_data.to_sql(f"player_{id}", con, if_exists="replace")
    player_data.to_sql(f"player_{id}", con, if_exists="append")
    
 


con.commit()


con.close()


Already Collected(2, 1983, 1996, 2, None)
Already Collected(3, 1988, 2002, 2, None)
Already Collected(7, 1981, 1998, 2, None)
Already Collected(9, 1983, 1996, 2, None)
Already Collected(12, 1993, 1998, 2, None)
Already Collected(15, 1994, 2007, 2, None)
Already Collected(17, 1983, 1997, 2, None)
Already Collected(21, 1991, 2001, 2, None)
Already Collected(22, 1988, 1999, 2, None)
Already Collected(23, 1986, 1999, 2, None)
Already Collected(24, 1992, 1994, 2, None)
Already Collected(26, 1991, 2000, 2, None)
Already Collected(28, 1989, 2000, 2, None)
Already Collected(29, 1994, 2000, 2, None)
Already Collected(30, 1992, 1995, 2, None)
Already Collected(31, 1992, 1999, 2, None)
Already Collected(32, 1994, 1997, 2, None)
Already Collected(35, 1993, 1998, 2, None)
Already Collected(36, 1989, 1996, 2, None)
Already Collected(37, 1993, 1997, 2, None)
Already Collected(38, 1987, 1997, 2, None)
Already Collected(41, 1990, 2002, 2, None)
Already Collected(42, 1994, 2002, 2, None)
Already Collect

In [55]:
## DONE Fill in Date Last Queried Field
con = sqlite3.connect("Data/CommonPlayerInfo.sqlite")

cur = con.cursor()


cur.execute("SELECT PERSON_ID, FROM_YEAR, TO_YEAR, GamesQueried, DateLastQueried FROM CommonPlayerInfo")
player_info = cur.fetchall()

for row in player_info:
        person_id = row[0]
        table_name = f"player_{person_id}"
        
        cur.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
        table_exists = cur.fetchone()

        if table_exists:
        # If the table exists, fetch the max(GAME_DATE)
                cur.execute(f"SELECT max(GAME_DATE) FROM {table_name}")
                date_test = cur.fetchone()[0]  # Get the first value from the result
                
                if date_test:
                        date_test = date_test[:10] # Take substring to remove T00:00:00 suffix from date values
                                                
                        #cur.execute(f"UPDATE CommonPlayerInfo SET DateLastQueried = {date_test} WHERE PERSON_ID = {row[0]}")
                        cur.execute("UPDATE CommonPlayerInfo SET DateLastQueried = ? WHERE PERSON_ID = ?", (date_test, person_id))
                        print(f"Updated Player {person_id} - Latest Game Date: {date_test}")
                else:
                        print(f"No game data found for Player {person_id}, skipping update.")
        else:
                print(f"Table {table_name} does not exist, skipping.")


con.commit()

con.close()


Updated Player 2 - Latest Game Date: 1997-05-10
Updated Player 3 - Latest Game Date: 2003-05-12
Updated Player 7 - Latest Game Date: 1999-05-13
Updated Player 9 - Latest Game Date: 1997-05-29
Updated Player 12 - Latest Game Date: 1999-02-28
Updated Player 15 - Latest Game Date: 2008-04-27
Updated Player 17 - Latest Game Date: 1998-05-03
Updated Player 21 - Latest Game Date: 2002-04-17
Updated Player 22 - Latest Game Date: 2000-06-19
Updated Player 23 - Latest Game Date: 2000-03-07
Updated Player 24 - Latest Game Date: 1995-04-23
Updated Player 26 - Latest Game Date: 2001-02-14
Updated Player 28 - Latest Game Date: 2000-12-20
Updated Player 29 - Latest Game Date: 2001-01-08
Updated Player 30 - Latest Game Date: 1996-03-25
Updated Player 31 - Latest Game Date: 2000-02-09
Updated Player 32 - Latest Game Date: 1997-11-21
Updated Player 35 - Latest Game Date: 1999-05-05
Updated Player 36 - Latest Game Date: 1996-12-03
Updated Player 37 - Latest Game Date: 1997-11-29
Updated Player 38 - Late

In [87]:
## Main Loop to update box score player game log db
## Master Game List
## Master Player List
## Master Game Log Table
## Individual Player game log tables

## TODO Check roster status.
## For active players, query games since DateLa stQueried (check for no dupe game id in player tables)
from nba_api.stats.endpoints import playergamelogs

## Connect to Database
con = sqlite3.connect("Data/CommonPlayerInfo.sqlite")

## Initialize cursor
cur = con.cursor()
## Generate list of active players 
cur.execute("SELECT PERSON_ID, ROSTERSTATUS, GamesQueried, DateLastQueried FROM CommonPlayerInfo WHERE ROSTERSTATUS == 'Active'")
player_info = cur.fetchall()

## Set Static Query Params
season = '2024-25'

## check date may need to subtract one day
date_now = datetime.now().date() - timedelta(days=1)
date_now = date_now.strftime("%m/%d/%Y")


## Loop through active players and query game logs since last query date
for row in player_info:

    ## Setting date that player logs were last queried for.
    ## Set to next day from DateLastQueried value to avoid duplicate entries.
    ## Set value to match format needed for nba_api PlayerGameLogs function params (MM/DD/YYYY)
    last_date = row[3]
    last_date = datetime.strptime(last_date, "%m/%d/%Y") + timedelta(days=1)
    last_date = last_date.strftime("%m/%d/%Y")
     
    
    id = row[0]
    print(id)
    print(date_now)
    print(last_date)
    player_log = playergamelogs.PlayerGameLogs(player_id_nullable=id, season_nullable= season, season_type_nullable="Regular Season",
                                               date_from_nullable=last_date,date_to_nullable=date_now)
    
    player_df = player_log.get_data_frames()[0]
    player_df['Season_Type'] = 'Regular Season'

    player_df.to_sql(f"player_{id}", con, if_exists="append")

    player_df_2 = player_df

    player_df_2["HOME_AWAY"] = ""
    player_df_2['Opp'] = ""
    
    i = 0
    for row in player_df_2["MATCHUP"]:
        
        ## Set Opp field
        player_df_2['Opp'][i] = player_df_2['MATCHUP'][i][-3:len(player_df_2['MATCHUP'][i])]
        print(i)

        ## Set HOME_AWAY Field
        if "@" in player_df_2['MATCHUP'][i]:
            player_df_2['HOME_AWAY'][i] = "Away"
            print("yes sir")
        else:
            player_df_2['HOME_AWAY'][i] = "Home"
            print("no mam")
        i+= 1
    

    player_df_2.to_sql("MasterGameLog", con, if_exists="append")

    cur.execute("UPDATE CommonPlayerInfo SET DateLastQueried = ? WHERE PERSON_ID = ?", (date_now,id))
    time.sleep(0.6)
   
    print(player_df.head())

con.commit()
con.close()

2544
02/22/2025
02/18/2025


C:\Users\mrsam\AppData\Local\Temp\ipykernel_14460\3633162604.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df_2['Opp'][i] = player_df_2['MATCHUP'][i][-3:len(player_df_2['MATCHUP'][i])]
C:\Users\mrsam\AppData\Local\Temp\ipykernel_14460\3633162604.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df_2['HOME_AWAY'][i] = "Away"
C:\Users\mrsam\AppData\Local\Temp\ipykernel_14460\3633162604.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

0
yes sir
1
yes sir
2
no mam
  SEASON_YEAR  PLAYER_ID   PLAYER_NAME NICKNAME     TEAM_ID TEAM_ABBREVIATION  \
0     2024-25       2544  LeBron James   LeBron  1610612747               LAL   
1     2024-25       2544  LeBron James   LeBron  1610612747               LAL   
2     2024-25       2544  LeBron James   LeBron  1610612747               LAL   

            TEAM_NAME     GAME_ID            GAME_DATE      MATCHUP  ...  \
0  Los Angeles Lakers  0022400808  2025-02-22T00:00:00    LAL @ DEN  ...   
1  Los Angeles Lakers  0022400796  2025-02-20T00:00:00    LAL @ POR  ...   
2  Los Angeles Lakers  0022400524  2025-02-19T00:00:00  LAL vs. CHA  ...   

  PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  \
0               1                     2         2         1   
1               2                     3         2         1   
2               3                     1         1         1   

   WNBA_FANTASY_PTS_RANK  AVAILABLE_FLAG  MIN_SEC     Season_Type  HOME_AWAY  \
0       

C:\Users\mrsam\AppData\Local\Temp\ipykernel_14460\3633162604.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df_2['Opp'][i] = player_df_2['MATCHUP'][i][-3:len(player_df_2['MATCHUP'][i])]
C:\Users\mrsam\AppData\Local\Temp\ipykernel_14460\3633162604.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df_2['HOME_AWAY'][i] = "Home"


0
no mam
1
no mam
  SEASON_YEAR  PLAYER_ID PLAYER_NAME NICKNAME     TEAM_ID TEAM_ABBREVIATION  \
0     2024-25     101108  Chris Paul    Chris  1610612759               SAS   
1     2024-25     101108  Chris Paul    Chris  1610612759               SAS   

           TEAM_NAME     GAME_ID            GAME_DATE      MATCHUP  ...  \
0  San Antonio Spurs  0022400802  2025-02-21T00:00:00  SAS vs. DET  ...   
1  San Antonio Spurs  0022400795  2025-02-20T00:00:00  SAS vs. PHX  ...   

  PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  \
0               2                     2         2         1   
1               1                     1         1         1   

   WNBA_FANTASY_PTS_RANK  AVAILABLE_FLAG  MIN_SEC     Season_Type  HOME_AWAY  \
0                      2               1    21:39  Regular Season       Home   
1                      1               1    31:53  Regular Season       Home   

   Opp  
0  DET  
1  PHX  

[2 rows x 72 columns]
200768
02/22/2025
02/18/2025
Empty Da

...

0
no mam
1
yes sir
  SEASON_YEAR  PLAYER_ID       PLAYER_NAME NICKNAME     TEAM_ID  \
0     2024-25    1641854  Craig Porter Jr.    Craig  1610612739   
1     2024-25    1641854  Craig Porter Jr.    Craig  1610612739   

  TEAM_ABBREVIATION            TEAM_NAME     GAME_ID            GAME_DATE  \
0               CLE  Cleveland Cavaliers  0022400797  2025-02-21T00:00:00   
1               CLE  Cleveland Cavaliers  0022400791  2025-02-20T00:00:00   

       MATCHUP  ... PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  \
0  CLE vs. NYK  ...               1                     1         1         1   
1    CLE @ BKN  ...               2                     2         1         1   

   WNBA_FANTASY_PTS_RANK  AVAILABLE_FLAG  MIN_SEC     Season_Type  HOME_AWAY  \
0                      1               1    12:00  Regular Season       Home   
1                      2               1     3:38  Regular Season       Away   

   Opp  
0  NYK  
1  BKN  

[2 rows x 72 columns]
1641857
02/2

C:\Users\mrsam\AppData\Local\Temp\ipykernel_14460\3633162604.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df_2['Opp'][i] = player_df_2['MATCHUP'][i][-3:len(player_df_2['MATCHUP'][i])]
C:\Users\mrsam\AppData\Local\Temp\ipykernel_14460\3633162604.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df_2['HOME_AWAY'][i] = "Home"


0
no mam
  SEASON_YEAR  PLAYER_ID PLAYER_NAME NICKNAME     TEAM_ID TEAM_ABBREVIATION  \
0     2024-25    1641871  Duop Reath     Duop  1610612757               POR   

                TEAM_NAME     GAME_ID            GAME_DATE      MATCHUP  ...  \
0  Portland Trail Blazers  0022400810  2025-02-22T00:00:00  POR vs. CHA  ...   

  PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  \
0               1                     1         1         1   

   WNBA_FANTASY_PTS_RANK  AVAILABLE_FLAG  MIN_SEC     Season_Type  HOME_AWAY  \
0                      1               1    10:34  Regular Season       Home   

   Opp  
0  CHA  

[1 rows x 72 columns]
1641998
02/22/2025
02/18/2025


C:\Users\mrsam\AppData\Local\Temp\ipykernel_14460\3633162604.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df_2['Opp'][i] = player_df_2['MATCHUP'][i][-3:len(player_df_2['MATCHUP'][i])]
C:\Users\mrsam\AppData\Local\Temp\ipykernel_14460\3633162604.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df_2['HOME_AWAY'][i] = "Away"


0
yes sir
1
yes sir
  SEASON_YEAR  PLAYER_ID       PLAYER_NAME NICKNAME     TEAM_ID  \
0     2024-25    1641998  Trey Jemison III     Trey  1610612747   
1     2024-25    1641998  Trey Jemison III     Trey  1610612747   

  TEAM_ABBREVIATION           TEAM_NAME     GAME_ID            GAME_DATE  \
0               LAL  Los Angeles Lakers  0022400808  2025-02-22T00:00:00   
1               LAL  Los Angeles Lakers  0022400796  2025-02-20T00:00:00   

     MATCHUP  ... PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  \
0  LAL @ DEN  ...               2                     2         1         1   
1  LAL @ POR  ...               1                     1         1         1   

   WNBA_FANTASY_PTS_RANK  AVAILABLE_FLAG  MIN_SEC     Season_Type  HOME_AWAY  \
0                      2               1     3:19  Regular Season       Away   
1                      1               1    14:11  Regular Season       Away   

   Opp  
0  DEN  
1  POR  

[2 rows x 72 columns]


In [85]:
# TODO DONE
## Configure last_date and date_now variables for game log date querying

## Test code segment for configuring dates to query game logs

season = '2024-25'
date_now = datetime.now().date()
date_now = date_now.strftime("%m/%d/%Y")
con = sqlite3.connect("Data/CommonPlayerInfo.sqlite")

cur = con.cursor()
cur.execute("SELECT PERSON_ID, ROSTERSTATUS, GamesQueried, DateLastQueried FROM CommonPlayerInfo WHERE ROSTERSTATUS == 'Active' AND PERSON_ID == '203999'")
player_info = cur.fetchall()

print(player_info)

id = player_info[0]
last_date = player_info[0][3]

last_date = datetime.strptime(last_date, "%m/%d/%Y")
last_date = last_date.strftime("%m/%d/%Y")


player_log = playergamelogs.PlayerGameLogs(player_id_nullable=id, season_nullable= season, season_type_nullable="Regular Season",
                                               date_from_nullable=last_date,date_to_nullable=date_now)

player_df = player_log.get_data_frames()[0]

player_df['HOME_AWAY'] = ''
player_df['Opp'] = ''

#print(player_df.head())
i = 0
for row in player_df["MATCHUP"]:
    
    #print(player_df['MATCHUP'][i][-3:len(player_df['MATCHUP'][i])])
    player_df['Opp'][i] = player_df['MATCHUP'][i][-3:len(player_df['MATCHUP'][i])]
    print(i)

    if "@" in player_df['MATCHUP'][i]:
        player_df['HOME_AWAY'][i] = "Away"
        print("yes sir")
    else:
        player_df['HOME_AWAY'][i] = "Home"
        print("no mam")
    i+= 1


print(player_df.head())
#print(player_df['MATCHUP'][0][-3:len(player_df['MATCHUP'][0])])
#print(player_df['MATCHUP'][1][-3:len(player_df['MATCHUP'][1])])

con.close()

[(203999, 'Active', 2, '02/17/2025')]
0
no mam
1
no mam
  SEASON_YEAR  PLAYER_ID   PLAYER_NAME NICKNAME     TEAM_ID TEAM_ABBREVIATION  \
0     2024-25     203999  Nikola Jokić   Nikola  1610612743               DEN   
1     2024-25     203999  Nikola Jokić   Nikola  1610612743               DEN   

        TEAM_NAME     GAME_ID            GAME_DATE      MATCHUP  ... PTS_RANK  \
0  Denver Nuggets  0022400808  2025-02-22T00:00:00  DEN vs. LAL  ...        2   
1  Denver Nuggets  0022400794  2025-02-20T00:00:00  DEN vs. CHA  ...        1   

   PLUS_MINUS_RANK  NBA_FANTASY_PTS_RANK  DD2_RANK  TD3_RANK  \
0                2                     2         1         1   
1                1                     1         1         2   

   WNBA_FANTASY_PTS_RANK  AVAILABLE_FLAG  MIN_SEC  HOME_AWAY  Opp  
0                      2               1    35:10       Home  LAL  
1                      1               1    36:55       Home  CHA  

[2 rows x 71 columns]


C:\Users\mrsam\AppData\Local\Temp\ipykernel_14460\826695348.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df['Opp'][i] = player_df['MATCHUP'][i][-3:len(player_df['MATCHUP'][i])]
C:\Users\mrsam\AppData\Local\Temp\ipykernel_14460\826695348.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df['HOME_AWAY'][i] = "Home"


In [ ]:
# TODO DONE
# ## One off code to concat indvidual player log tables into master game log sheet




# # Connect to the database
# con = sqlite3.connect("Data/CommonPlayerInfo.sqlite")
# cur = con.cursor()

# # Fetch a sample player table name to copy the schema
# cur.execute("SELECT PERSON_ID FROM CommonPlayerInfo LIMIT 1")
# sample_player = cur.fetchone()

# if sample_player:
#     sample_table = f"player_{sample_player[0]}"
    
#     # Get schema from sample player table
#     cur.execute(f"PRAGMA table_info({sample_table})")
#     columns = cur.fetchall()

#     # Filter column definitions (exclude primary keys, etc.)
#     column_definitions = ", ".join([f'"{col[1]}" {col[2]}' for col in columns if col[1].lower() != "id"])  
#     column_definitions = f'"PERSON_ID" INTEGER, {column_definitions}'

#     # Create MasterGameLog if it doesn't exist
#     cur.execute(f"CREATE TABLE IF NOT EXISTS MasterGameLog ({column_definitions})")

#     # Fetch all PERSON_IDs
#     cur.execute("SELECT PERSON_ID FROM CommonPlayerInfo")
#     player_ids = cur.fetchall()

#     for row in player_ids:
#         person_id = row[0]
#         table_name = f"player_{person_id}"

#         # Check if the table exists
#         cur.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
#         if cur.fetchone():
#             print(f"Processing {table_name}...")

#             # Get column names dynamically (excluding index-related fields)
#             cur.execute(f"PRAGMA table_info({table_name})")
#             table_columns = [col[1] for col in cur.fetchall()]
#             table_columns_str = ", ".join([f'"{col}"' for col in table_columns])

#             # Insert data from player table into MasterGameLog
#             cur.execute(f'INSERT INTO MasterGameLog ("PERSON_ID", {table_columns_str}) SELECT {person_id}, * FROM {table_name}')

#     # Commit changes
#     con.commit()
#     print("MasterGameLog updated successfully!")

# else:
#     print("No player tables found in database.")

# # Close connection
# con.close()


Processing player_2...
Processing player_3...
Processing player_7...
Processing player_9...
Processing player_12...
Processing player_15...
Processing player_17...
Processing player_21...
Processing player_22...
Processing player_23...
Processing player_24...
Processing player_26...
Processing player_28...
Processing player_29...
Processing player_30...
Processing player_31...
Processing player_32...
Processing player_35...
Processing player_36...
Processing player_37...
Processing player_38...
Processing player_41...
Processing player_42...
Processing player_43...
Processing player_45...
Processing player_46...
Processing player_47...
Processing player_49...
Processing player_51...
Processing player_52...
Processing player_53...
Processing player_54...
Processing player_55...
Processing player_56...
Processing player_57...
Processing player_61...
Processing player_63...
Processing player_64...
Processing player_65...
Processing player_66...
Processing player_67...
Processing player_70

In [ ]:
## TODO DONE

# Add column to game log to denote home vs away and call out opponenet 

# Connect to the database
# con = sqlite3.connect("Data/CommonPlayerInfo.sqlite")
# cur = con.cursor()

# # Ensure the columns exist in MasterGameLog
# cur.execute("PRAGMA table_info(MasterGameLog)")
# existing_columns = [col[1] for col in cur.fetchall()]

# if "HOME_AWAY" not in existing_columns:
#     cur.execute("ALTER TABLE MasterGameLog ADD COLUMN HOME_AWAY TEXT")

# if "Opp" not in existing_columns:
#     cur.execute("ALTER TABLE MasterGameLog ADD COLUMN Opp TEXT")

# # Update the entire table at once
# cur.execute("""
#     UPDATE MasterGameLog
#     SET HOME_AWAY = CASE 
#         WHEN MATCHUP LIKE '%@%' THEN 'Away' 
#         ELSE 'Home' 
#     END,
#     Opp = SUBSTR(MATCHUP, -3)
# """)

# # Commit changes
# con.commit()
# print("HOME_AWAY and Opp columns updated successfully!")

# # Close connection
# con.close()



HOME_AWAY and Opp columns updated successfully!


In [19]:
## TODO Join Advanced box score stats to game logs 

## Get List of Game IDs (should be reproducable for when season starts back up in october)
## Create table in DB for game IDs

from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
team_df = pd.DataFrame(nba_teams)
games_master = pd.DataFrame()


## USING THIS AS MASTER LIST OF GAMES PLAYED NEED TO EXCLUDE SL GAMES
## Pulls summer league games as well as playoffs
for team_id in team_df['id']:
    games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_type_nullable= 'Regular Season' )
    games_df = games.get_data_frames()[0]
    games_master = pd.concat([games_master,games_df], ignore_index=True)
    time.sleep(0.6)

## Adding label to differentiate Summer League, Regular Season and Playoffs
games_master['GameType'] = 'Regular Season'
games_master.loc[games_master['GAME_ID'].str.startswith(('15', '16', '13')), 'GameType'] = 'Summer League'

playoffs = pd.DataFrame()

for team_id in team_df['id']:
    games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_type_nullable= 'Playoffs' )
    games_df = games.get_data_frames()[0]
    playoffs = pd.concat([playoffs,games_df], ignore_index=True)
    time.sleep(0.6)

playoffs['GameType'] = 'Playoffs'

## Joining Playoffs and Regular Season/Summer League Dataframes 
games_master = pd.concat([games_master,playoffs], ignore_index=True)


## Adding this table to Database 
con = sqlite3.connect("Data/test_adv.sqlite")
games_master.to_sql("MasterGameList", con, if_exists="replace")
con.close()


In [ ]:
## TODO Join Advanced box score stats to game logs 

## Loop Design 
## Define game logs needed to be queried based on date last queried.  


id = 203999
#id = 203076


test= playergamelogs.PlayerGameLogs(player_id_nullable = id, season_nullable= season, season_type_nullable='Regular Season')
player_season = test.get_data_frames()[0]

print(player_season.head())




In [2]:
## TODO Define Advanced Data to Pull From Available Sheets

## Setup parmas to use for example queries
id = 203999 #ID is for Nikola Jokic
season = '2024-25'
game_id = '0022400341'



from nba_api.stats.endpoints import boxscoreadvancedv3
from nba_api.stats.endpoints import boxscoredefensivev2
from nba_api.stats.endpoints import boxscorefourfactorsv3
from nba_api.stats.endpoints import boxscorehustlev2
from nba_api.stats.endpoints import boxscorematchupsv3
from nba_api.stats.endpoints import boxscoremiscv3
from nba_api.stats.endpoints import boxscoreplayertrackv3
from nba_api.stats.endpoints import boxscorescoringv3
from nba_api.stats.endpoints import boxscoresummaryv2
from nba_api.stats.endpoints import boxscoretraditionalv3
from nba_api.stats.endpoints import boxscoreusagev3


In [7]:
query_1 = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=game_id)
query_2 = boxscoredefensivev2.BoxScoreDefensiveV2(game_id=game_id)
query_3 = boxscorefourfactorsv3.BoxScoreFourFactorsV3(game_id=game_id)
query_4 = boxscorehustlev2.BoxScoreHustleV2(game_id=game_id)
query_5 = boxscorematchupsv3.BoxScoreMatchupsV3(game_id=game_id)
query_6 = boxscoremiscv3.BoxScoreMiscV3(game_id=game_id)
query_7 = boxscoreplayertrackv3.BoxScorePlayerTrackV3(game_id=game_id)
query_8 = boxscorescoringv3.BoxScoreScoringV3(game_id=game_id)
query_9 = boxscoresummaryv2.BoxScoreSummaryV2(game_id=game_id)
query_10 = boxscoretraditionalv3.BoxScoreTraditionalV3(game_id=game_id)
query_11 = boxscoreusagev3.BoxScoreUsageV3(game_id=game_id)




queries =[query_1,
          query_2,
          query_3,
          query_4,
          query_5,
          query_6,
          query_7,
          query_8,
          query_9,
          query_10,
          query_11]


query_name_1 = "BoxScoreAdvancedV3"
query_name_2 = "BoxScoreDefensiveV2"
query_name_3 = "BoxScoreFourFactorsV3"
query_name_4 = "BoxScoreHustleV2"
query_name_5 = "BoxScoreMatchupsV3"
query_name_6 = "BoxScoreMiscV3"
query_name_7 = "BoxScorePlayerTrackV3"
query_name_8 = "BoxScoreScoringV3"
query_name_9 = "BoxScoreSummaryV2"
query_name_10 = "BoxScoreTraditionalV3"
query_name_11 = "BoxScoreUsageV3"




query_names =[query_name_1,
          query_name_2,
          query_name_3,
          query_name_4,
          query_name_5,
          query_name_6,
          query_name_7,
          query_name_8,
          query_name_9,
          query_name_10,
          query_name_11]

In [ ]:
## One off loop to save copies of all raw advanced stats tables

## 11 "Box Score"  variants were chosen that explore additional game factors
for i in range(0,11):
    

    for j in range(0,10):
        try:
            df = queries[i].get_data_frames()[j]
            print(df.head())
            file_path = f"Data/Advanced Stats Examples/{query_names[i]}_df_{j}.csv"
            df.to_csv(file_path,index=False)
        

    
        except:
            print("No More Dataframes")
            break



    print(query_names[i])


       gameId      teamId teamCity teamName teamTricode teamSlug  personId  \
0  0022400341  1610612743   Denver  Nuggets         DEN  nuggets   1631212   
1  0022400341  1610612743   Denver  Nuggets         DEN  nuggets   1629008   
2  0022400341  1610612743   Denver  Nuggets         DEN  nuggets    203999   
3  0022400341  1610612743   Denver  Nuggets         DEN  nuggets   1631128   
4  0022400341  1610612743   Denver  Nuggets         DEN  nuggets    201566   

   firstName  familyName          nameI  ... turnoverRatio  \
0     Peyton      Watson      P. Watson  ...          12.5   
1    Michael  Porter Jr.  M. Porter Jr.  ...           8.7   
2     Nikola       Jokić       N. Jokić  ...           8.8   
3  Christian       Braun       C. Braun  ...          12.5   
4    Russell   Westbrook   R. Westbrook  ...           9.1   

  effectiveFieldGoalPercentage trueShootingPercentage usagePercentage  \
0                        0.400                  0.400           0.098   
1           

In [ ]:
## BoxScoreAdvancedV3 EDA

from nba_api.stats.endpoints import boxscoreadvancedv3



#  "Data/Advanced Stats Examples/.csv"


query_1 = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=game_id)
query_name = "BoxScoreAdvancedV3"

for i in range(0,10):
    try:
        df_1 = query_1.get_data_frames()[i]
        print(df_1.head())
        file_path = f"Data/Advanced Stats Examples/{query_name}_df_{i}.csv"
        

    
    except:
        print("No More Dataframes")
        break




       gameId      teamId teamCity  ... pacePer40 possessions    PIE
0  0022400341  1610612743   Denver  ...     93.20        59.0  0.017
1  0022400341  1610612743   Denver  ...     89.54        83.0  0.055
2  0022400341  1610612743   Denver  ...     89.52        87.0  0.231
3  0022400341  1610612743   Denver  ...     88.14        83.0  0.025
4  0022400341  1610612743   Denver  ...     91.51        74.0  0.116

[5 rows x 37 columns]
Data/Advanced Stats Examples/BoxScoreAdvancedV3_df_0.csv
       gameId      teamId    teamCity  ... pacePer40 possessions    PIE
0  0022400341  1610612764  Washington  ...     87.08       104.0  0.526
1  0022400341  1610612743      Denver  ...     87.08       105.0  0.474

[2 rows x 30 columns]
Data/Advanced Stats Examples/BoxScoreAdvancedV3_df_1.csv
No More Dataframes
